# Pysle - a python interface to the ISLEX dictionary

*Pronunciationtools*

In this tutorial, we'll talk about the high-level features offered by `pronunciationtools`, one of the submodules of `Pysle`.  If this is your first exposure to Pysle, you'll want to checkout the [Intro to Psyle tutorial](https://nbviewer.jupyter.org/github/timmahrt/pysle/blob/main/tutorials/tutorial1_intro_to_pysle.ipynb) first.

Pronunciationtools offers four different tool that can be accessed fairly easily. If you're ok getting your hands dirty, you can get the same functionality by digging into the `isletool` and `phonetics` submodules.  The four tools are:
- [alignPronunciations](#alignpronunciations)
- [findClosestEntryForPhones](#findclosestentryforphones)
- [findClosestEntryForSyllabification](#findclosestentryforsyllabification)
- [findBestSyllabification](#findbestsyllabification)

These tools are generally used to work with pronunciation data that you already have and to use the isle dictionary in conjunction with it somehow.

Before getting started, let's warm up our dev environment:

In [43]:
from pysle import isletool
from pysle import pronunciationtools

isle = isletool.Isle()

Text


## alignPronunciations

alignPronunciations is a tool for mapping some pronunciation to pronunciations from the pysle dictionary.

Suppose we have some recordings from a production experiment.  Someone meticulously annotated the phonemes for the recordings. But now we want to label the syllables and identify which syllables are stressed. Pysle can be used to automatically guess syllable stress placement. However, the phonemes in the dictionary pronunications and in the actual pronunications may differ.

In order to make a mapping between the two, we can use alignPronunciations.  And in fact, its what is used internally by some other tools we'll look at.

In [44]:
word = "anxious"

entries = isle.lookup(word)
dictionaryPronunciation1 = entries[0].syllabificationList[0].desyllabify().phonemes

dictionaryPronunciation2 = entries[1].syllabificationList[0].desyllabify().phonemes

print(dictionaryPronunciation1)
print(dictionaryPronunciation2)

['ˈæ', 'ŋ', 'ʃ', 'ə', 's']
['ˈæ', 'ŋ', 'k', 'ʃ', 'ə', 's']


In [45]:
actualPronunciation = ['ˈæ', 'ŋ', 'k', 'tʃ', 'ə', 's'] # Hyper articulation?

actualAlignedWithFirst, firstAlignedWithActual = pronunciationtools.alignPronunciations(actualPronunciation, dictionaryPronunciation1, True)
print(actualAlignedWithFirst.phonemes)
print(firstAlignedWithActual.phonemes)

['ˈæ', 'ŋ', 'k', 'tʃ', 'ə', 's']
['ˈæ', 'ŋ', 'ʃ', "''", 'ə', 's']


In [46]:
actualPronunciation = ['ˈæ', 'ŋ', 'k', 'tʃ', 'ə', 's']

actualAlignedWithSecond, secondAlignedWithActual = pronunciationtools.alignPronunciations(actualPronunciation, dictionaryPronunciation2, True)
print(actualAlignedWithSecond.phonemes)
print(secondAlignedWithActual.phonemes)

['ˈæ', 'ŋ', 'k', 'tʃ', 'ə', 's']
['ˈæ', 'ŋ', 'k', 'ʃ', 'ə', 's']


alignPronunciations finds the longest non-continuous sequence (eg ['ˈæ', 'ŋ', 'k', 'ə', 's'] in the last example) and then transforms the two phone lists along that sequence.  In the spot where the two pronunciations differed, one had 'tʃ' and the other 'ʃ'--the function basically tolerates any phone to fill that position.

Let's try again but with a constant against a vowel:

In [47]:
dictionaryPronunciation = ['ˈæ', 'ŋ', 'k', 'ʃ', 'ə', 's']
fakePronunciation = ['ˈæ', 'ŋ', 'k', 'o', 'ə', 's']

actualAlignedWithSecond, secondAlignedWithActual = pronunciationtools.alignPronunciations(dictionaryPronunciation, fakePronunciation, True)
print(actualAlignedWithSecond.phonemes)
print(secondAlignedWithActual.phonemes)

['ˈæ', 'ŋ', 'k', 'ʃ', 'ə', "''", 's']
['ˈæ', 'ŋ', 'k', "''", 'o', 'ə', 's']


Since fuzzy matching is set to True, it paired the shwa with 'o' (eg ['ˈæ', 'ŋ', 'k', 'V', 's'] where 'V' fills in for 'shwa' and 'o'), This left an unmatched 'ʃ' in the first word and an unmatched 'shwa' in the second word.  

Fuzzy matching will allow any vowel to match with any other vowel. It will also match nasals together and match rhotics together.

Let's try again but with fuzzy matching set to False.

In [48]:
dictionaryPronunciation = ['t', 'ˈæ', 'ŋ', 'k', 'ʃ', 'ə', 's']
fakePronunciation = ['ˈæ', 'ŋ', 'k', 'o', 'o', 'ə', 's', 't']

actualAlignedWithSecond, secondAlignedWithActual = pronunciationtools.alignPronunciations(dictionaryPronunciation, fakePronunciation, False)
print(actualAlignedWithSecond.phonemes)
print(secondAlignedWithActual.phonemes)

['t', 'ˈæ', 'ŋ', 'k', 'ʃ', "''", 'ə', 's', "''"]
["''", 'ˈæ', 'ŋ', 'k', 'o', 'o', 'ə', 's', 't']


This time matching was strict (ie ['ˈæ', 'ŋ', 'k', 'ə', 's']). After matching is done, align will pair unrelated phones and then insert spaces where appropriate to make the two strings the same length.

## findClosestEntryForPhones

If we have a word and a list of phones `findClosestEntryForPhones` will return the entry that is most similar. To determine similarity, it will use the same logic as `alignPronunciations` and then count how many changes are needed to align the two.

Let's start by taking a look at some entries for a word in the dictionary:

In [49]:
results = isle.lookup('another')
for entry in results:
    print(entry.toList())

[[['ə'], ['n', 'ˈʌ'], ['ð', 'ɚ']]]
[[['ə'], ['n', 'ˈʌ'], ['ð', 'ə', 'ɹ']]]


If we lookup a word that has a pronunciation that doesn't match either of those, pysle will return the one that matches best.

In [50]:
word = "another"
pronunciation = ['ei', 'n', 'ˈʌ', 'ð', 'ɚ']
closestEntry = pronunciationtools.findClosestEntryForPhones(isle, word, pronunciation)
print(closestEntry.toList())

[[['ə'], ['n', 'ˈʌ'], ['ð', 'ɚ']]]


## findClosestEntryForSyllabification

`findClosestEntryForPhones` has a close relative in `findClosestEntryForSyllabification`. However, the syllabification version has a bit of a different response.  Instead of just finding the closest entry, it also returns the constructed version of the input that was used to find the closest entry.

In [51]:
word = "another"
pronunciation = [['ei'], ['ð', 'ɚ']]
closestEntry, constructedEntry = pronunciationtools.findClosestEntryForSyllabification(isle, word, pronunciation)
print(closestEntry.toList())
print(constructedEntry.toList())

[[['ə'], ['n', 'ˈʌ'], ['ð', 'ɚ']]]
[[['ei'], ["''", "''", 'ð', 'ɚ']]]


## findBestSyllabification

If we have a word and a list of phones, findBestSyllabification will give us the best syllabification for those phones. It does this by 
1) finding all entries for the word
2) finding the phone list from those entries that closest matches the provided phone list
3) lining up the provided phone list and matched phone list
4) and finally mapping the matched phone list's syllable structure onto the provided phone list

Let's take a look at an example:

In [52]:
word = "another"
pronunciation = ['ei', 'n', 'ˈʌ', 'ð', 'ɚ']
bestSyllabification = pronunciationtools.findBestSyllabification(isle, word, pronunciation)
print(bestSyllabification.toList())

[['ei'], ['n', 'ˈʌ'], ['ð', 'ɚ']]


The system easily stumbles though, so its best to use with clean data.  If a syllable gets dropped, for example, the results probably won't be very meaningful if there isn't a similar entry in ISLEX.

In [53]:
word = "another"
pronunciation = ['n', 'ˈʌ', 'ð', 'ɚ']
bestSyllabification = pronunciationtools.findBestSyllabification(isle, word, pronunciation)
print(bestSyllabification.toList())

[['n'], ['ˈʌ', 'ð'], ['ɚ']]
